<a href="https://colab.research.google.com/github/hijera/NN_colab_hw/blob/main/%D0%94%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D0%B5%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_%D0%BF%D0%BE_%D1%82%D0%B5%D0%BC%D0%B5_%C2%AB%D0%A0%D0%B5%D0%BA%D0%BE%D0%BC%D0%B5%D0%BD%D0%B4%D0%B0%D1%86%D0%B8%D0%B8_%D0%BD%D0%B0_%D0%BE%D1%81%D0%BD%D0%BE%D0%B2%D0%B5_%D1%81%D0%BE%D0%B4%D0%B5%D1%80%D0%B6%D0%B0%D0%BD%D0%B8%D1%8F%C2%BB_(%D0%92%D0%BB%D0%B0%D1%81%D0%BE%D0%B2_%D0%90%D0%BD%D1%82%D0%BE%D0%BD)_jupyter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.  Использовать датасет MovieLens.
2.  Построить рекомендации (регрессия, предсказываем оценку) на фичах:

*  TF-IDF на тегах и жанрах;
*  средние оценки (+ median, variance и т. д.) пользователя и фильма.

    Оценить RMSE на тестовой выборке.



In [ ]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.pipeline import Pipeline

%matplotlib inline

Загружаем датасет

In [ ]:
!wget  https://files.grouplens.org/datasets/movielens/ml-latest-small.zip -O ml-latest.zip
#!wget https://files.grouplens.org/datasets/movielens/ml-latest.zip -O ml-latest.zip
!unzip -o ml-latest.zip
#!rm ./ml-latest -Rf
#!mv ./ml-latest-small ./ml-latest
prefix = 'ml-latest-small'

--2023-12-02 07:57:23--  https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: 'ml-latest.zip'

     0K .......... .......... .......... .......... ..........  5%  178K 5s
    50K .......... .......... .......... .......... .......... 10%  359K 4s
   100K .......... .......... .......... .......... .......... 15% 13,8M 2s
   150K .......... .......... .......... .......... .......... 20%  182K 3s
   200K .......... .......... .......... .......... .......... 26% 97,5M 2s
   250K .......... .......... .......... .......... .......... 31%  297M 2s
   300K .......... .......... .......... .......... .......... 36% 5,21M 1s
   350K .......... .......... .......... .......... .......... 41%  973K 1s
   400K .......... ......

Archive:  ml-latest.zip
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [ ]:


#links = pd.read_csv(os.path.join(prefix, 'links.csv'))
movies = pd.read_csv(os.path.join(prefix, 'movies.csv'))
ratings = pd.read_csv(os.path.join(prefix, 'ratings.csv'))
tags = pd.read_csv(os.path.join(prefix, 'tags.csv')).fillna("")
#genome_tags=pd.read_csv(os.path.join(prefix, 'genome-tags.csv'))
#genome_scores=pd.read_csv(os.path.join(prefix, 'genome-scores.csv'))

In [ ]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [ ]:
tags=tags.drop(['userId'],axis=1)
tags


,movieId,tag,timestamp
0,60756,funny,1445714994
1,60756,Highly quotable,1445714996
2,60756,will ferrell,1445714992
3,89774,Boxing story,1445715207
4,89774,MMA,1445715200
...,...,...,...
3678,7382,for katie,1171234019
3679,7936,austere,1173392334
3680,3265,gun fu,1493843984
3681,3265,heroic bloodshed,1493843978


In [ ]:
tags['grouped_tags']=tags.groupby('movieId')['tag'].transform(lambda x: '|'.join(x))
gtags=tags.copy()
gtags=gtags.drop(columns=['tag','timestamp']).drop_duplicates()
#Приводим тэги к нижнему регистру
gtags['grouped_tags']=gtags['grouped_tags'].str.lower()
gtags

,movieId,grouped_tags
0,60756,funny|highly quotable|will ferrell|comedy|funn...
3,89774,boxing story|mma|tom hardy
6,106782,drugs|leonardo dicaprio|martin scorsese|stock ...
9,48516,way too long|leonardo dicaprio|suspense|twist ...
10,431,al pacino|gangster|mafia
...,...,...
3674,1948,british
3676,5694,70mm
3677,6107,world war ii
3679,7936,austere


In [ ]:
#ratings=ratings.drop(columns=['userId'])
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [ ]:
#Соединяем данные
#joined_data = movies.join(ratings.set_index('movieId'), on='movieId')

In [ ]:
def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

movie_genres = joined_data.genres.apply(change_string).tolist()

Используем TF-IDF

In [ ]:
def min_max_scaling(series):
    return (series - series.min()) / (series.max() - series.min())

amrating = min_max_scaling(ratings.groupby('movieId')['rating'].mean()).to_frame().reset_index(drop=False).fillna(0.0).rename(columns = {'rating':'average_movie_rating'})
aurating  = min_max_scaling(ratings.groupby('userId')['rating'].mean()).to_frame().reset_index(drop=False).fillna(0.0).rename(columns = {'rating':'average_user_rating'})


#joined_data['average_movie_rating']=amrating.fillna(0.0);
#joined_data['average_user_rating']=aurating.fillna(0.0);
joined_ratings=ratings.join(amrating.set_index('movieId'),on='movieId')
joined_ratings=joined_ratings.join(aurating.set_index('userId'),on='userId')
joined_ratings

,userId,movieId,rating,timestamp,average_movie_rating,average_user_rating
0,1,1,4.0,964982703,0.760207,0.829900
1,1,3,4.0,964981247,0.613248,0.829900
2,1,6,4.0,964982224,0.765795,0.829900
3,1,47,5.0,964983815,0.772304,0.829900
4,1,50,5.0,964982931,0.830610,0.829900
...,...,...,...,...,...,...
100831,610,166534,4.0,1493848402,0.629630,0.647935
100832,610,168248,5.0,1493850091,0.809524,0.647935
100833,610,168250,5.0,1494273047,0.696296,0.647935
100834,610,168252,5.0,1493846352,0.840000,0.647935


In [ ]:
joined_data = movies.merge(joined_ratings, on='movieId')
joined_data=joined_data.join(gtags.set_index('movieId'),on='movieId',lsuffix="t_") #
joined_data=joined_data.fillna("")
joined_data

,movieId,title,genres,userId,rating,timestamp,average_movie_rating,average_user_rating,grouped_tags
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703,0.760207,0.829900,pixar|pixar|fun
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962,0.760207,0.633923,pixar|pixar|fun
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946,0.760207,0.524903,pixar|pixar|fun
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970,0.760207,0.583395,pixar|pixar|fun
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483,0.760207,0.787792,pixar|pixar|fun
...,...,...,...,...,...,...,...,...,...
100831,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,184,4.0,1537109082,0.777778,0.652409,
100832,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,184,3.5,1537109545,0.666667,0.652409,
100833,193585,Flint (2017),Drama,184,3.5,1537109805,0.666667,0.652409,
100834,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,184,3.5,1537110021,0.666667,0.652409,


In [ ]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(movie_genres)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

test = change_string("Adventure|Comedy|Fantasy|Crime")

predict = count_vect.transform([test])
X_tfidf2 = tfidf_transformer.transform(predict)

#pipe = Pipeline([('count', CountVectorizer()),
#                 ('tfid', TfidfTransformer())])
pipe= TfidfVectorizer()
vect_genres = TfidfVectorizer()
vect_tags = TfidfVectorizer()
tfidf_genres=vect_genres.fit_transform(movie_genres)
movie_tags=joined_data.grouped_tags.apply(change_string).tolist()
tfidf_tags=vect_tags.fit_transform(movie_tags)
#joined_data['genres_TFIDF'] = tfidf_genres.toarray().tolist()
#joined_data['tags_TFIDF'] = tfidf_tags.toarray().tolist()
joined_data['genres']=joined_data['genres'].str.lower()
joined_data.head(100)

,movieId,title,genres,userId,rating,timestamp,average_movie_rating,average_user_rating,grouped_tags
0,1,Toy Story (1995),adventure|animation|children|comedy|fantasy,1,4.0,964982703,0.760207,0.829900,pixar|pixar|fun
1,1,Toy Story (1995),adventure|animation|children|comedy|fantasy,5,4.0,847434962,0.760207,0.633923,pixar|pixar|fun
2,1,Toy Story (1995),adventure|animation|children|comedy|fantasy,7,4.5,1106635946,0.760207,0.524903,pixar|pixar|fun
3,1,Toy Story (1995),adventure|animation|children|comedy|fantasy,15,2.5,1510577970,0.760207,0.583395,pixar|pixar|fun
4,1,Toy Story (1995),adventure|animation|children|comedy|fantasy,17,4.5,1305696483,0.760207,0.787792,pixar|pixar|fun
...,...,...,...,...,...,...,...,...,...
95,1,Toy Story (1995),adventure|animation|children|comedy|fantasy,269,5.0,850865423,0.760207,0.629715,pixar|pixar|fun
96,1,Toy Story (1995),adventure|animation|children|comedy|fantasy,270,5.0,853918728,0.760207,0.516779,pixar|pixar|fun
97,1,Toy Story (1995),adventure|animation|children|comedy|fantasy,273,5.0,835861234,0.760207,0.731544,pixar|pixar|fun
98,1,Toy Story (1995),adventure|animation|children|comedy|fantasy,274,4.0,1171410158,0.760207,0.526412,pixar|pixar|fun


In [ ]:
genres_dense = tfidf_genres.todense().tolist() #.todense().tolist()
feature_genres = vect_genres.get_feature_names_out()
tags_dense = tfidf_tags.todense().tolist() # .todense().tolist()
feature_tags = vect_tags.get_feature_names_out()
df_genres = pd.DataFrame(genres_dense, columns=feature_genres)
df_tags = pd.DataFrame(tags_dense, columns=feature_tags)

In [ ]:
df_tags['worldwari'].unique()

array([0., 1.])

In [ ]:
y = joined_data[['rating']]
X =  pd.concat([joined_data['average_movie_rating'],joined_data['average_user_rating'],df_tags,df_genres], axis=1)
X=X.fillna(0.0)
X

,average_movie_rating,average_user_rating,06oscarnominatedbestmovieanimation,1900s,1920s,1950s,1960s,1970s,1980s,1990s,...,horror,imax,musical,mystery,nogenreslisted,romance,scifi,thriller,war,western
0,0.760207,0.829900,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.760207,0.633923,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.760207,0.524903,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.821009,0.0,0.0,0.0,0.0
3,0.760207,0.583395,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.726241,0.0,0.0,0.0,0.0
4,0.760207,0.787792,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100831,0.777778,0.652409,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
100832,0.666667,0.652409,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
100833,0.666667,0.652409,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
100834,0.666667,0.652409,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [ ]:
y

,rating
0,4.0
1,4.0
2,4.5
3,2.5
4,4.5
...,...
100831,4.0
100832,3.5
100833,3.5
100834,3.5


In [ ]:
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_test.index


Int64Index([67037, 42175, 93850,  6187, 12229,  7433, 53802, 65098, 68041,
            11854,
            ...
            33873, 42562,   210, 62777, 85948, 33213, 75004, 58134, 33261,
            63946],
           dtype='int64', length=30251)

In [ ]:
kmodel = KNeighborsRegressor(5, n_jobs=-1)
model=LinearRegression()
res=kmodel.fit(X_train, y_train)

In [ ]:
print("SCORE:"+str(kmodel.score(X_test,y_test)))
y_pred=kmodel.predict(X_test)
print("rmse:"+str(np.sqrt(mean_squared_error(y_test,y_pred))))
print("mse: "+str(mean_squared_error(y_test,y_pred)))


SCORE:0.2830812069990494
rmse:0.8878310790859426
mse: 0.7882440249909093
